In [68]:
from datasets import load_dataset, Dataset, load_from_disk
import pandas as pd
from transformers import pipeline
import unicodedata

Select toxic comments, pass them through detox model, get detoxed comments and augument them with their toxic counterparts

In [30]:
#!/bin/bash
!curl -L -o ./youtube-toxicity-data.zip\
  https://www.kaggle.com/api/v1/datasets/download/reihanenamdari/youtube-toxicity-data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 99167  100 99167    0     0  47747      0  0:00:02  0:00:02 --:--:--  158k


In [32]:
!rm -rf yt
!unzip youtube-toxicity-data.zip

Archive:  youtube-toxicity-data.zip
  inflating: youtoxic_english_1000.csv  


In [33]:
!ls

DatasetPrep.ipynb  SBIC		include  pyvenv.cfg
LICENSE		   SBIC.v2.tgz	lib	 youtoxic_english_1000.csv
README.md	   bin		lib64	 youtube-toxicity-data.zip


In [21]:
train_ds=load_dataset("csv", data_files={"train": "SBIC/SBIC.v2.trn.csv"}, delimiter=",")
train_ds

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['whoTarget', 'intentYN', 'sexYN', 'sexReason', 'offensiveYN', 'annotatorGender', 'annotatorMinority', 'sexPhrase', 'speakerMinorityYN', 'WorkerId', 'HITId', 'annotatorPolitics', 'annotatorRace', 'annotatorAge', 'post', 'targetMinority', 'targetCategory', 'targetStereotype', 'dataSource'],
        num_rows: 112900
    })
})

In [22]:
train_ds["train"].unique("offensiveYN")

[1.0, 0.5, 0.0, None]

In [34]:
df=pd.read_csv("youtoxic_english_1000.csv")

In [39]:
df.head(5)

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [40]:
new_df=df[df["IsToxic"]==True]
new_df.describe()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
count,462,462,462,462,462,462,462,462,462,462,462,462,462,462,462
unique,462,13,459,1,2,2,2,2,2,2,2,2,1,2,1
top,Ugifh2DMhBbDkHgCoAEC,04kJtp6pVXI,run them over,True,True,False,False,False,False,False,False,False,False,False,False
freq,1,111,3,462,353,441,301,362,324,337,454,461,462,450,462


In [44]:
new_df.count()

CommentId          462
VideoId            462
Text               462
IsToxic            462
IsAbusive          462
IsThreat           462
IsProvocative      462
IsObscene          462
IsHatespeech       462
IsRacist           462
IsNationalist      462
IsSexist           462
IsHomophobic       462
IsReligiousHate    462
IsRadicalism       462
dtype: int64

In [45]:
df=new_df[["Text", "IsToxic"]]

In [47]:
df.head(1)

,Text,IsToxic
1,Law enforcement is not trained to shoot to app...,True


In [50]:
ds=Dataset.from_pandas(df, preserve_index=False)

In [61]:
def normalize_unicode_data(ex_batch):
    return {
        "Text": [unicodedata.normalize("NFKD", ex) for ex in ex_batch["Text"]]
    }

ds=ds.map(lambda ex_batch: normalize_unicode_data(ex_batch), batched=True)

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

In [62]:
ds

Dataset({
    features: ['Text', 'IsToxic'],
    num_rows: 462
})

In [63]:
ds[0]

{'Text': 'Law enforcement is not trained to shoot to apprehend.  They are trained to shoot to kill.  And I thank Wilson for killing that punk bitch.',
 'IsToxic': True}

In [64]:
ds.save_to_disk("yt-toxic-ds")

Saving the dataset (0/1 shards):   0%|          | 0/462 [00:00<?, ? examples/s]

In [70]:
ds=load_from_disk("./yt-toxic-ds")
ds

Dataset({
    features: ['Text', 'IsToxic'],
    num_rows: 462
})